In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json

import os

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [3]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [4]:
file_parameters

{'watershed': 'han'}

In [5]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [6]:
pd.set_option('display.max_columns', 1000)

In [7]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [8]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [9]:
gain_epochs , rnn_epochs

(2000, 150)

In [10]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

In [11]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])

    df_all, train_mean, train_std, df = normalize(df)



    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/han/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['가평_2016.xlsx', '가평_2017.xlsx', '가평_2018.xlsx', '가평_2019.xlsx'], ['서상_2016.xlsx', '서상_2017.xlsx', '서상_2018.xlsx', '서상_2019.xlsx'], ['의암호_2016.xlsx', '의암호_2017.xlsx', '의암호_2018.xlsx', '의암호_2019.xlsx']]
data/han/자동/가평_2016.xlsx
data/han/자동/가평_2017.xlsx
data/han/자동/가평_2018.xlsx
data/han/자동/가평_2019.xlsx
time range in files :  2016-01-01 00:00  ~  2019-12-31 23:00
data/han/자동/서상_2016.xlsx
data/han/자동/서상_2017.xlsx
data/han/자동/서상_2018.xlsx
data/han/자동/서상_2019.xlsx
time range in files :  2016-01-01 00:00  ~  2019-12-31 23:00
data/han/자동/의암호_2016.xlsx
data/han/자동/의암호_2017.xlsx
data/han/자동/의암호_2018.xlsx
data/han/자동/의암호_2019.xlsx
time range in files :  2016-01-01 00:00  ~  2019-12-31 23:00
MissData :  save/  miss :  (5327, 12)
MissData :  save/  miss :  (5327, 12)
Epoch 1/2000
1/1 [==============================] - 0s 193ms/step - gen_loss: 151.1350 - disc_loss: 0.7173 - rmse: 1.25

1/1 [==============================] - 0s 20ms/step - gen_loss: 12.2329 - disc_loss: 0.3542 - rmse: 0.4317 - val_loss: 0.3552
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.9420 - disc_loss: 0.3422 - rmse: 0.4591 - val_loss: 0.3743
Epoch 55/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 14.7227 - disc_loss: 0.3332 - rmse: 0.4136 - val_loss: 0.3762
Epoch 56/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.4075 - disc_loss: 0.3477 - rmse: 0.4236 - val_loss: 0.3580
Epoch 57/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.9130 - disc_loss: 0.3574 - rmse: 0.4157 - val_loss: 0.3642
Epoch 58/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.6234 - disc_loss: 0.3409 - rmse: 0.4265 - val_loss: 0.3518
Epoch 59/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.8893 - disc_loss: 0.3366 - rmse: 0.3441 - val_loss: 0.3542
Epoch 60/2000
1/1 [==================

Epoch 112/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.3728 - disc_loss: 0.2764 - rmse: 0.3802 - val_loss: 0.3308
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8062 - disc_loss: 0.2839 - rmse: 0.3070 - val_loss: 0.3139
Epoch 114/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 14.9130 - disc_loss: 0.2867 - rmse: 0.3051 - val_loss: 0.4209
Epoch 115/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.4592 - disc_loss: 0.2782 - rmse: 0.3830 - val_loss: 0.3757
Epoch 116/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.6457 - disc_loss: 0.2821 - rmse: 0.3390 - val_loss: 0.3354
Epoch 117/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.2163 - disc_loss: 0.2831 - rmse: 0.3251 - val_loss: 0.3791
Epoch 118/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.0164 - disc_loss: 0.2733 - rmse: 0.3630 - val_loss: 0.3232
Epoch 119/2000
1/1

1/1 [==============================] - 0s 20ms/step - gen_loss: 14.8118 - disc_loss: 0.2487 - rmse: 0.3646 - val_loss: 0.3043
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2299 - disc_loss: 0.2393 - rmse: 0.2786 - val_loss: 0.3313
Epoch 172/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.4599 - disc_loss: 0.2563 - rmse: 0.3656 - val_loss: 0.2656
Epoch 173/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.0728 - disc_loss: 0.2420 - rmse: 0.3374 - val_loss: 0.2603
Epoch 174/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6564 - disc_loss: 0.2520 - rmse: 0.2722 - val_loss: 0.2958
Epoch 175/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6525 - disc_loss: 0.2423 - rmse: 0.3406 - val_loss: 0.3280
Epoch 176/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0413 - disc_loss: 0.2513 - rmse: 0.3058 - val_loss: 0.3158
Epoch 177/2000
1/1 [==============

1/1 [==============================] - 0s 21ms/step - gen_loss: 7.8158 - disc_loss: 0.2356 - rmse: 0.3593 - val_loss: 0.3409
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.1540 - disc_loss: 0.2424 - rmse: 0.3144 - val_loss: 0.2849
Epoch 230/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9513 - disc_loss: 0.2444 - rmse: 0.2840 - val_loss: 0.2824
Epoch 231/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.1310 - disc_loss: 0.2372 - rmse: 0.3172 - val_loss: 0.3035
Epoch 232/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.4625 - disc_loss: 0.2363 - rmse: 0.3516 - val_loss: 0.3085
Epoch 233/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.2183 - disc_loss: 0.2211 - rmse: 0.3618 - val_loss: 0.3016
Epoch 234/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0071 - disc_loss: 0.2410 - rmse: 0.2965 - val_loss: 0.2975
Epoch 235/2000
1/1 [===============

...

1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6919 - disc_loss: 0.0916 - rmse: 0.0848 - val_loss: 0.1012
Epoch 1767/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 0.7172 - disc_loss: 0.0934 - rmse: 0.0738 - val_loss: 0.1270
Epoch 1768/2000
1/1 [==============================] - ETA: 0s - gen_loss: 0.6175 - disc_loss: 0.0897 - rmse: 0.09 - 0s 20ms/step - gen_loss: 0.6175 - disc_loss: 0.0897 - rmse: 0.0907 - val_loss: 0.0668
Epoch 1769/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 0.6064 - disc_loss: 0.0948 - rmse: 0.0504 - val_loss: 0.0577
Epoch 1770/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 0.6470 - disc_loss: 0.0990 - rmse: 0.0694 - val_loss: 0.0583
Epoch 1771/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 0.5935 - disc_loss: 0.0975 - rmse: 0.0537 - val_loss: 0.0626
Epoch 1772/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 0.6042 - disc_loss: 0.0996 -

1/1 [==============================] - 0s 19ms/step - gen_loss: 0.5844 - disc_loss: 0.0882 - rmse: 0.0484 - val_loss: 0.0594
Epoch 1824/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6702 - disc_loss: 0.0946 - rmse: 0.0677 - val_loss: 0.0603
Epoch 1825/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6673 - disc_loss: 0.0937 - rmse: 0.1381 - val_loss: 0.0466
Epoch 1826/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6579 - disc_loss: 0.0932 - rmse: 0.0819 - val_loss: 0.0696
Epoch 1827/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.5887 - disc_loss: 0.0913 - rmse: 0.0616 - val_loss: 0.0836
Epoch 1828/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6454 - disc_loss: 0.0929 - rmse: 0.0785 - val_loss: 0.0657
Epoch 1829/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6313 - disc_loss: 0.0953 - rmse: 0.1137 - val_loss: 0.0634
Epoch 1830/2000
1/1 [========

1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7614 - disc_loss: 0.0883 - rmse: 0.1045 - val_loss: 0.0727
Epoch 1882/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7166 - disc_loss: 0.0993 - rmse: 0.0604 - val_loss: 0.0689
Epoch 1883/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6411 - disc_loss: 0.0899 - rmse: 0.0929 - val_loss: 0.0674
Epoch 1884/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6779 - disc_loss: 0.0975 - rmse: 0.1107 - val_loss: 0.0635
Epoch 1885/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6614 - disc_loss: 0.0996 - rmse: 0.0728 - val_loss: 0.0806
Epoch 1886/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6197 - disc_loss: 0.0946 - rmse: 0.0778 - val_loss: 0.0633
Epoch 1887/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 0.6664 - disc_loss: 0.0925 - rmse: 0.0825 - val_loss: 0.0692
Epoch 1888/2000
1/1 [========

1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6490 - disc_loss: 0.0870 - rmse: 0.0651 - val_loss: 0.0771
Epoch 1940/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7826 - disc_loss: 0.0840 - rmse: 0.1134 - val_loss: 0.1115
Epoch 1941/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6982 - disc_loss: 0.0973 - rmse: 0.0690 - val_loss: 0.0851
Epoch 1942/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7853 - disc_loss: 0.0911 - rmse: 0.0640 - val_loss: 0.0658
Epoch 1943/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7316 - disc_loss: 0.0929 - rmse: 0.0726 - val_loss: 0.0715
Epoch 1944/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7184 - disc_loss: 0.0954 - rmse: 0.0637 - val_loss: 0.0551
Epoch 1945/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6795 - disc_loss: 0.0903 - rmse: 0.0706 - val_loss: 0.1142
Epoch 1946/2000
1/1 [========

data/han/퇴적물/청평댐2_2016.xlsx
data/han/퇴적물/청평댐2_2017.xlsx
data/han/퇴적물/청평댐2_2018.xlsx
data/han/퇴적물/청평댐2_2019.xlsx
time range in files :  2016-01-01 00:00  ~  2019-12-31 23:00
data/han/퇴적물/춘성교_2016.xlsx
data/han/퇴적물/춘성교_2017.xlsx
data/han/퇴적물/춘성교_2018.xlsx
data/han/퇴적물/춘성교_2019.xlsx
time range in files :  2016-01-01 00:00  ~  2019-12-31 23:00
data/han/퇴적물/의암댐2_2016.xlsx
data/han/퇴적물/의암댐2_2017.xlsx
data/han/퇴적물/의암댐2_2018.xlsx
data/han/퇴적물/의암댐2_2019.xlsx
time range in files :  2016-01-01 00:00  ~  2019-12-31 23:00
data/han/퇴적물/춘천댐2_2016.xlsx
data/han/퇴적물/춘천댐2_2017.xlsx
data/han/퇴적물/춘천댐2_2018.xlsx
data/han/퇴적물/춘천댐2_2019.xlsx
time range in files :  2016-01-01 00:00  ~  2019-12-31 23:00
data/han/퇴적물/춘천댐3_2016.xlsx
data/han/퇴적물/춘천댐3_2017.xlsx
data/han/퇴적물/춘천댐3_2018.xlsx
data/han/퇴적물/춘천댐3_2019.xlsx
time range in files :  2016-01-01 00:00  ~  2019-12-31 23:00
interpol flag :  [True, True]
folder :  data/han/조류/
colum_idx :  :,[30,31,36,40,50]
file_names[idx] :  [['의암호_2016.xlsx', '의암호_2017.xlsx',

In [12]:
print(real_df_all.shape)

(35064, 530)


In [13]:
train_df, val_df, test_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)

In [14]:
print('-------------------prediction')
print('-------------------prediction')
print('-------------------prediction')

print('real_df_all.type : ', type(real_df_all))
print('train_df.type : ', type(train_df))
print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)


-------------------prediction
-------------------prediction
-------------------prediction
real_df_all.type :  <class 'pandas.core.frame.DataFrame'>
train_df.type :  <class 'pandas.core.frame.DataFrame'>
train_df.shape :  (28051, 530) val_df.shape :  (3506, 530) test_df.shape: (3507, 530)


In [15]:

label_columns_indices = {name: i for i, name in enumerate(dfff[0])}

print("label_columns_indices:")
print(label_columns_indices)


target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

print('target columns : ', rnn_target_column)
num_features = dfff[0].shape[1]

target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)


label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target columns :  do
target_col_idx :  2
out_num_features :  1


In [16]:
val_nse = {}
val_pbias = {}


WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

if __RNN_TRAINING__:
    if not os.path.exists('save/' + watershed):
        os.makedirs('save/' + watershed)


idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]

indices = {name: i for i, name in enumerate(idx)}

model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
print("save model path : ", model_path)

val_nse = {}
val_pbias = {}

 # +"gru.ckpt" -- path




save model path :  save/han/models/do/


## 모델 학습

In [17]:
multi_linear_model = model_multi_linear(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    #training_flag=__RNN_TRAINING__, checkpoint_path="save/"+watershed+"models/multi_linear.ckpt")
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_linear.ckpt")
elman_model = model_elman(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"elman.ckpt")
gru_model = model_gru(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")
multi_lstm_model = model_multi_lstm(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_lstm.ckpt")
multi_conv_model = model_multi_conv(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_conv.ckpt")


Epoch 1/150
1/1 [==============================] - 1s 1s/step - loss: 0.6682 - mean_absolute_error: 0.6715 - val_loss: 0.4157 - val_mean_absolute_error: 0.4708
Epoch 2/150
1/1 [==============================] - 1s 972ms/step - loss: 0.5060 - mean_absolute_error: 0.6054 - val_loss: 0.3508 - val_mean_absolute_error: 0.4423
Epoch 3/150
1/1 [==============================] - 1s 1s/step - loss: 0.4089 - mean_absolute_error: 0.5074 - val_loss: 0.2989 - val_mean_absolute_error: 0.3971
Epoch 4/150
1/1 [==============================] - 1s 1s/step - loss: 0.3142 - mean_absolute_error: 0.4347 - val_loss: 0.2028 - val_mean_absolute_error: 0.3253
Epoch 5/150
1/1 [==============================] - 1s 1s/step - loss: 0.2250 - mean_absolute_error: 0.3652 - val_loss: 0.1413 - val_mean_absolute_error: 0.2693
Epoch 6/150
1/1 [==============================] - 1s 1s/step - loss: 0.1531 - mean_absolute_error: 0.3000 - val_loss: 0.0807 - val_mean_absolute_error: 0.2167
Epoch 7/150
1/1 [====================

1/1 [==============================] - 1s 1s/step - loss: 0.0785 - mean_absolute_error: 0.2201 - val_loss: 0.1326 - val_mean_absolute_error: 0.2961
Epoch 52/150
1/1 [==============================] - 1s 1s/step - loss: 0.0811 - mean_absolute_error: 0.2122 - val_loss: 0.1137 - val_mean_absolute_error: 0.2719
Epoch 53/150
1/1 [==============================] - 1s 1s/step - loss: 0.0740 - mean_absolute_error: 0.2078 - val_loss: 0.1533 - val_mean_absolute_error: 0.3050
Epoch 54/150
1/1 [==============================] - 1s 1s/step - loss: 0.0661 - mean_absolute_error: 0.1965 - val_loss: 0.1276 - val_mean_absolute_error: 0.2872
Epoch 55/150
1/1 [==============================] - 1s 1s/step - loss: 0.0677 - mean_absolute_error: 0.1985 - val_loss: 0.1265 - val_mean_absolute_error: 0.2862
Epoch 56/150
1/1 [==============================] - 1s 955ms/step - loss: 0.0715 - mean_absolute_error: 0.2046 - val_loss: 0.1072 - val_mean_absolute_error: 0.2670
Epoch 57/150
1/1 [==========================

1/1 [==============================] - 1s 974ms/step - loss: 0.0614 - mean_absolute_error: 0.1856 - val_loss: 0.1239 - val_mean_absolute_error: 0.2887
Epoch 102/150
1/1 [==============================] - 1s 975ms/step - loss: 0.0695 - mean_absolute_error: 0.2004 - val_loss: 0.1155 - val_mean_absolute_error: 0.2790
Epoch 103/150
1/1 [==============================] - 1s 967ms/step - loss: 0.0571 - mean_absolute_error: 0.1822 - val_loss: 0.1134 - val_mean_absolute_error: 0.2644
Epoch 104/150
1/1 [==============================] - 1s 1s/step - loss: 0.0627 - mean_absolute_error: 0.1843 - val_loss: 0.1254 - val_mean_absolute_error: 0.2883
Epoch 105/150
1/1 [==============================] - 1s 1s/step - loss: 0.0474 - mean_absolute_error: 0.1690 - val_loss: 0.1271 - val_mean_absolute_error: 0.2860
Epoch 106/150
1/1 [==============================] - 1s 1s/step - loss: 0.0763 - mean_absolute_error: 0.2055 - val_loss: 0.1213 - val_mean_absolute_error: 0.2872
Epoch 107/150
1/1 [==============

1/1 [==============================] - 1s 1s/step - loss: 0.6594 - mean_absolute_error: 0.6708 - val_loss: 0.5073 - val_mean_absolute_error: 0.5575
Epoch 2/150
1/1 [==============================] - 1s 963ms/step - loss: 0.7457 - mean_absolute_error: 0.6966 - val_loss: 0.5038 - val_mean_absolute_error: 0.5502
Epoch 3/150
1/1 [==============================] - 1s 981ms/step - loss: 0.6405 - mean_absolute_error: 0.6696 - val_loss: 0.3759 - val_mean_absolute_error: 0.4576
Epoch 4/150
1/1 [==============================] - 1s 971ms/step - loss: 0.5758 - mean_absolute_error: 0.6376 - val_loss: 0.3063 - val_mean_absolute_error: 0.3992
Epoch 5/150
1/1 [==============================] - 1s 980ms/step - loss: 0.4397 - mean_absolute_error: 0.5371 - val_loss: 0.2532 - val_mean_absolute_error: 0.3557
Epoch 6/150
1/1 [==============================] - 1s 965ms/step - loss: 0.2981 - mean_absolute_error: 0.4203 - val_loss: 0.1698 - val_mean_absolute_error: 0.2749
Epoch 7/150
1/1 [====================

1/1 [==============================] - 1s 950ms/step - loss: 0.0749 - mean_absolute_error: 0.1978 - val_loss: 0.0836 - val_mean_absolute_error: 0.2311
Epoch 52/150
1/1 [==============================] - 1s 976ms/step - loss: 0.0567 - mean_absolute_error: 0.1812 - val_loss: 0.0816 - val_mean_absolute_error: 0.2297
Epoch 53/150
1/1 [==============================] - 1s 987ms/step - loss: 0.0647 - mean_absolute_error: 0.1956 - val_loss: 0.0941 - val_mean_absolute_error: 0.2377
Epoch 54/150
1/1 [==============================] - 1s 958ms/step - loss: 0.0707 - mean_absolute_error: 0.2002 - val_loss: 0.0819 - val_mean_absolute_error: 0.2222
Epoch 55/150
1/1 [==============================] - 1s 973ms/step - loss: 0.0658 - mean_absolute_error: 0.1878 - val_loss: 0.0990 - val_mean_absolute_error: 0.2428
Epoch 56/150
1/1 [==============================] - 1s 976ms/step - loss: 0.0661 - mean_absolute_error: 0.1888 - val_loss: 0.1033 - val_mean_absolute_error: 0.2570
Epoch 57/150
1/1 [===========

...

1/1 [==============================] - 1s 1s/step - loss: 0.0662 - mean_absolute_error: 0.1911 - val_loss: 0.1240 - val_mean_absolute_error: 0.2785
Epoch 53/150
1/1 [==============================] - 1s 955ms/step - loss: 0.0547 - mean_absolute_error: 0.1663 - val_loss: 0.0994 - val_mean_absolute_error: 0.2494
Epoch 54/150
1/1 [==============================] - 1s 947ms/step - loss: 0.0488 - mean_absolute_error: 0.1646 - val_loss: 0.1264 - val_mean_absolute_error: 0.2852
Epoch 55/150
1/1 [==============================] - 1s 1s/step - loss: 0.0561 - mean_absolute_error: 0.1765 - val_loss: 0.1017 - val_mean_absolute_error: 0.2549
Epoch 56/150
1/1 [==============================] - 1s 1s/step - loss: 0.0597 - mean_absolute_error: 0.1761 - val_loss: 0.1265 - val_mean_absolute_error: 0.2868
Epoch 57/150
1/1 [==============================] - 1s 1s/step - loss: 0.0656 - mean_absolute_error: 0.1848 - val_loss: 0.1106 - val_mean_absolute_error: 0.2635
Epoch 58/150
1/1 [=======================

1/1 [==============================] - 1s 944ms/step - loss: 0.0460 - mean_absolute_error: 0.1540 - val_loss: 0.0967 - val_mean_absolute_error: 0.2442
Epoch 103/150
1/1 [==============================] - 1s 1s/step - loss: 0.0414 - mean_absolute_error: 0.1486 - val_loss: 0.0754 - val_mean_absolute_error: 0.2109
Epoch 104/150
1/1 [==============================] - 1s 1s/step - loss: 0.0442 - mean_absolute_error: 0.1539 - val_loss: 0.0825 - val_mean_absolute_error: 0.2291
Epoch 105/150
1/1 [==============================] - 1s 1s/step - loss: 0.0403 - mean_absolute_error: 0.1449 - val_loss: 0.0825 - val_mean_absolute_error: 0.2297
Epoch 106/150
1/1 [==============================] - 1s 1s/step - loss: 0.0421 - mean_absolute_error: 0.1493 - val_loss: 0.0839 - val_mean_absolute_error: 0.2317
Epoch 107/150
1/1 [==============================] - 1s 1s/step - loss: 0.0397 - mean_absolute_error: 0.1392 - val_loss: 0.0702 - val_mean_absolute_error: 0.2075
Epoch 108/150
1/1 [====================

1/1 [==============================] - 1s 1s/step - loss: 0.4865 - mean_absolute_error: 0.5810 - val_loss: 0.1687 - val_mean_absolute_error: 0.3248
Epoch 3/150
1/1 [==============================] - 1s 1s/step - loss: 0.3199 - mean_absolute_error: 0.4443 - val_loss: 0.1568 - val_mean_absolute_error: 0.3189
Epoch 4/150
1/1 [==============================] - 1s 1s/step - loss: 0.3535 - mean_absolute_error: 0.4639 - val_loss: 0.1210 - val_mean_absolute_error: 0.2610
Epoch 5/150
1/1 [==============================] - 1s 1s/step - loss: 0.1583 - mean_absolute_error: 0.3126 - val_loss: 0.3595 - val_mean_absolute_error: 0.4572
Epoch 6/150
1/1 [==============================] - 1s 1s/step - loss: 0.2361 - mean_absolute_error: 0.3607 - val_loss: 0.4365 - val_mean_absolute_error: 0.5506
Epoch 7/150
1/1 [==============================] - 1s 1s/step - loss: 0.2553 - mean_absolute_error: 0.3821 - val_loss: 0.4108 - val_mean_absolute_error: 0.5279
Epoch 8/150
1/1 [==============================] - 1

Epoch 53/150
1/1 [==============================] - 1s 997ms/step - loss: 0.0598 - mean_absolute_error: 0.1764 - val_loss: 0.0901 - val_mean_absolute_error: 0.2356
Epoch 54/150
1/1 [==============================] - 1s 970ms/step - loss: 0.0582 - mean_absolute_error: 0.1813 - val_loss: 0.1018 - val_mean_absolute_error: 0.2428
Epoch 55/150
1/1 [==============================] - 1s 998ms/step - loss: 0.0623 - mean_absolute_error: 0.1857 - val_loss: 0.0857 - val_mean_absolute_error: 0.2263
Epoch 56/150
1/1 [==============================] - 1s 967ms/step - loss: 0.0402 - mean_absolute_error: 0.1501 - val_loss: 0.0967 - val_mean_absolute_error: 0.2388
Epoch 57/150
1/1 [==============================] - 1s 1s/step - loss: 0.0493 - mean_absolute_error: 0.1678 - val_loss: 0.1253 - val_mean_absolute_error: 0.2774
Epoch 58/150
1/1 [==============================] - 1s 968ms/step - loss: 0.0476 - mean_absolute_error: 0.1660 - val_loss: 0.0849 - val_mean_absolute_error: 0.2263
Epoch 59/150
1/1 [=

1/1 [==============================] - 1s 1s/step - loss: 0.0464 - mean_absolute_error: 0.1621 - val_loss: 0.0757 - val_mean_absolute_error: 0.2087
Epoch 104/150
1/1 [==============================] - 1s 1s/step - loss: 0.0425 - mean_absolute_error: 0.1575 - val_loss: 0.0854 - val_mean_absolute_error: 0.2204
Epoch 105/150
1/1 [==============================] - 1s 969ms/step - loss: 0.0509 - mean_absolute_error: 0.1626 - val_loss: 0.0785 - val_mean_absolute_error: 0.2193
Epoch 106/150
1/1 [==============================] - 1s 1s/step - loss: 0.0325 - mean_absolute_error: 0.1370 - val_loss: 0.0840 - val_mean_absolute_error: 0.2203
Epoch 107/150
1/1 [==============================] - 1s 1s/step - loss: 0.0337 - mean_absolute_error: 0.1398 - val_loss: 0.0888 - val_mean_absolute_error: 0.2299
Epoch 108/150
1/1 [==============================] - 1s 1s/step - loss: 0.0409 - mean_absolute_error: 0.1474 - val_loss: 0.0889 - val_mean_absolute_error: 0.2347
Epoch 109/150
1/1 [====================

## core / window.py / 

In [18]:
%matplotlib widget

In [44]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    print(df.shape)
    print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(length):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    print(inputs.shape)
    print(labels.shape)

    predictions = model(inputs)
    print(predictions.shape)
    
    predictions = predictions.numpy() * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    #plt.figure(figsize=(10, 800))
    
    input_index = np.array(range(length))
    label_index = input_index + 24*7
    
    print(input_index)
    print(label_index)
    
    plt.figure()
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, 0, :], label='label')
    plt.plot(label_index, pred_day[:, 0, :], label='pred')
    plt.legend()
    #plt.show()
    #for i in range(200):
        #plt.subplot(200, 1, i+1)
        #plt.plot([0,1,2,3,4,5,6],inputs_day[i, :, :], label='input', marker='.', zorder=-10, c='#ff00FF')
        #plt.plot([7,8,9,10,11],label_day[i, :, :], label='label', marker='.', zorder=-10, c='#2ca02c')
        #plt.plot([7,8,9,10,11],pred_day[i, :, :], label='pred', marker='.', zorder=-10, c='#ff7f0e')
        #plt.show
    
    
    
    print(pred_day.shape)
    print(label_day.shape)
    
    o1 = np.mean(labels)
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    return nse3, np.abs(pbias3), pred_day, labels
    

In [45]:
val_nse['Linear'], val_pbias['Linear'], pred, label = compa(
    model=gru_model,df=test_df,
    plot_col=out_features[0], target_std=target_std, target_mean=target_mean, predict_day = rnn_predict_day)

print()
print()

print(val_nse['Linear'])
print(val_pbias['Linear'])
#print(pred)
#print(label)

(3507, 530)
2
(3219, 168, 530)
(3219, 120, 1)
(3219, 120, 1)
[   0    1    2 ... 3216 3217 3218]
[ 168  169  170 ... 3384 3385 3386]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(3219, 5, 1)
(3219, 5, 1)


[0.80439765]
[1.27010442]


In [40]:
ssss = test_df.iloc[:,out_features[0]].to_numpy() * target_std[out_features[0]] + target_mean[out_features[0]]

In [41]:
plt.figure()
plt.plot(ssss)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
val_nse['Linear'], val_pbias['Linear'], pred, label = multi_window.compa(
     multi_linear_model, plot_col=out_features[0], windows=multi_window.example3,
     target_std=target_std, target_mean=target_mean, predict_day = rnn_predict_day)

val_nse['Linear'], val_pbias['Linear']

(0.6209511035137023, 3.5729259301029352)

In [22]:
val_nse['Linear'], val_pbias['Linear'], pred, label = multi_window.compa(
     multi_linear_model, plot_col=out_features[0], windows=multi_window.example3,
     target_std=target_std, target_mean=target_mean, predict_day = rnn_predict_day)
val_nse['ELMAN'], val_pbias['ELMAN'], pred, label = multi_window.compa(
     elman_model, plot_col=out_features[0], windows=multi_window.example3,
     target_std=target_std, target_mean=target_mean, predict_day = rnn_predict_day)
val_nse['GRU'], val_pbias['GRU'], pred, label = multi_window.compa(
     gru_model, plot_col=out_features[0], windows=multi_window.example3,
     target_std=target_std, target_mean=target_mean, predict_day = rnn_predict_day)
val_nse['LSTM'], val_pbias['LSTM'], pred, label = multi_window.compa(
     multi_lstm_model, plot_col=out_features[0], windows=multi_window.example3,
     target_std=target_std, target_mean=target_mean, predict_day = rnn_predict_day)
val_nse['CONV'], val_pbias['CONV'], pred, label = multi_window.compa(
     multi_conv_model, plot_col=out_features[0], windows=multi_window.example3,
     target_std=target_std, target_mean=target_mean, predict_day = rnn_predict_day)

In [23]:
print("save model path : ", model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('tatget : ', rnn_target_column)
print('target col index : ', target_col_idx)
print('Linear : ', val_nse['Linear'], val_pbias['Linear'])
print('ELMAN : ', val_nse['ELMAN'], val_pbias['ELMAN'])
print('GRU : ', val_nse['GRU'], val_pbias['GRU'])
print('LSTM : ', val_nse['LSTM'], val_pbias['LSTM'])
print('CNN : ', val_nse['CONV'], val_pbias['CONV'])
print('GAIN_VAL_PER : ', gain_val_performance)
print('GAIN_TEST_PER : ', gain_performance)

save model path :  save/han/models/do/
year : 2016 ~ 2019
run :  range(0, 9)
tatget :  do
target col index :  2
Linear :  0.597472443229311 4.288270139304241
ELMAN :  0.7707641499705061 0.7448832317923405
GRU :  0.8036432626299967 1.8578509119966193
LSTM :  0.8325209676654177 0.15671626023638394
CNN :  0.7043353965041259 0.17237030315391894
GAIN_VAL_PER :  {'0': 0.23652982711791992, '4': 0.40480589866638184, '5': 0.09700687229633331}
GAIN_TEST_PER :  {'0': 0.26513025164604187, '4': 0.3079327642917633, '5': 0.09512912482023239}


In [24]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …